# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f0ba4206f40>
├── 'a' --> tensor([[-0.7937, -0.4952, -0.4200],
│                   [-0.3142, -0.3004,  1.2702]])
└── 'x' --> <FastTreeValue 0x7f0ba4206e20>
    └── 'c' --> tensor([[ 0.4012,  0.4930, -0.6113, -0.6209],
                        [-0.9880,  0.4415, -1.6825,  1.4976],
                        [-0.5949, -0.7458,  0.3910,  0.3924]])

In [4]:
t.a

tensor([[-0.7937, -0.4952, -0.4200],
        [-0.3142, -0.3004,  1.2702]])

In [5]:
%timeit t.a

58.4 ns ± 0.0649 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f0ba4206f40>
├── 'a' --> tensor([[ 0.0430, -0.4567,  0.0439],
│                   [-0.8979, -0.8304,  0.1905]])
└── 'x' --> <FastTreeValue 0x7f0ba4206e20>
    └── 'c' --> tensor([[ 0.4012,  0.4930, -0.6113, -0.6209],
                        [-0.9880,  0.4415, -1.6825,  1.4976],
                        [-0.5949, -0.7458,  0.3910,  0.3924]])

In [7]:
%timeit t.a = new_value

60.3 ns ± 0.0215 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.7937, -0.4952, -0.4200],
               [-0.3142, -0.3004,  1.2702]]),
    x: Batch(
           c: tensor([[ 0.4012,  0.4930, -0.6113, -0.6209],
                      [-0.9880,  0.4415, -1.6825,  1.4976],
                      [-0.5949, -0.7458,  0.3910,  0.3924]]),
       ),
)

In [10]:
b.a

tensor([[-0.7937, -0.4952, -0.4200],
        [-0.3142, -0.3004,  1.2702]])

In [11]:
%timeit b.a

57 ns ± 0.0111 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.2739,  2.9644,  1.9805],
               [ 0.5534, -0.6204,  1.4132]]),
    x: Batch(
           c: tensor([[ 0.4012,  0.4930, -0.6113, -0.6209],
                      [-0.9880,  0.4415, -1.6825,  1.4976],
                      [-0.5949, -0.7458,  0.3910,  0.3924]]),
       ),
)

In [13]:
%timeit b.a = new_value

478 ns ± 0.185 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

842 ns ± 5.13 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.2 µs ± 38 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 314 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 195 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f0aeca69bb0>
├── 'a' --> tensor([[[-0.7937, -0.4952, -0.4200],
│                    [-0.3142, -0.3004,  1.2702]],
│           
│                   [[-0.7937, -0.4952, -0.4200],
│                    [-0.3142, -0.3004,  1.2702]],
│           
│                   [[-0.7937, -0.4952, -0.4200],
│                    [-0.3142, -0.3004,  1.2702]],
│           
│                   [[-0.7937, -0.4952, -0.4200],
│                    [-0.3142, -0.3004,  1.2702]],
│           
│                   [[-0.7937, -0.4952, -0.4200],
│                    [-0.3142, -0.3004,  1.2702]],
│           
│                   [[-0.7937, -0.4952, -0.4200],
│                    [-0.3142, -0.3004,  1.2702]],
│           
│                   [[-0.7937, -0.4952, -0.4200],
│                    [-0.3142, -0.3004,  1.2702]],
│           
│                   [[-0.7937, -0.4952, -0.4200],
│                    [-0.3142, -0.3004,  1.2702]]])
└── 'x' --> <FastTreeValue 0x7f0aec44ffa0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.9 µs ± 117 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f0af30ae880>
├── 'a' --> tensor([[-0.7937, -0.4952, -0.4200],
│                   [-0.3142, -0.3004,  1.2702],
│                   [-0.7937, -0.4952, -0.4200],
│                   [-0.3142, -0.3004,  1.2702],
│                   [-0.7937, -0.4952, -0.4200],
│                   [-0.3142, -0.3004,  1.2702],
│                   [-0.7937, -0.4952, -0.4200],
│                   [-0.3142, -0.3004,  1.2702],
│                   [-0.7937, -0.4952, -0.4200],
│                   [-0.3142, -0.3004,  1.2702],
│                   [-0.7937, -0.4952, -0.4200],
│                   [-0.3142, -0.3004,  1.2702],
│                   [-0.7937, -0.4952, -0.4200],
│                   [-0.3142, -0.3004,  1.2702],
│                   [-0.7937, -0.4952, -0.4200],
│                   [-0.3142, -0.3004,  1.2702]])
└── 'x' --> <FastTreeValue 0x7f0af30ae9d0>
    └── 'c' --> tensor([[ 0.4012,  0.4930, -0.6113, -0.6209],
                        [-0.9880,  0.4415, -1.6825,  1.4976],
                 

In [23]:
%timeit t_cat(trees)

29.6 µs ± 16 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.8 µs ± 197 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.4012,  0.4930, -0.6113, -0.6209],
                       [-0.9880,  0.4415, -1.6825,  1.4976],
                       [-0.5949, -0.7458,  0.3910,  0.3924]],
              
                      [[ 0.4012,  0.4930, -0.6113, -0.6209],
                       [-0.9880,  0.4415, -1.6825,  1.4976],
                       [-0.5949, -0.7458,  0.3910,  0.3924]],
              
                      [[ 0.4012,  0.4930, -0.6113, -0.6209],
                       [-0.9880,  0.4415, -1.6825,  1.4976],
                       [-0.5949, -0.7458,  0.3910,  0.3924]],
              
                      [[ 0.4012,  0.4930, -0.6113, -0.6209],
                       [-0.9880,  0.4415, -1.6825,  1.4976],
                       [-0.5949, -0.7458,  0.3910,  0.3924]],
              
                      [[ 0.4012,  0.4930, -0.6113, -0.6209],
                       [-0.9880,  0.4415, -1.6825,  1.4976],
                       [-0.5949, -0.7458,  0.3910,  0.3924]],

In [26]:
%timeit Batch.stack(batches)

78.2 µs ± 128 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.4012,  0.4930, -0.6113, -0.6209],
                      [-0.9880,  0.4415, -1.6825,  1.4976],
                      [-0.5949, -0.7458,  0.3910,  0.3924],
                      [ 0.4012,  0.4930, -0.6113, -0.6209],
                      [-0.9880,  0.4415, -1.6825,  1.4976],
                      [-0.5949, -0.7458,  0.3910,  0.3924],
                      [ 0.4012,  0.4930, -0.6113, -0.6209],
                      [-0.9880,  0.4415, -1.6825,  1.4976],
                      [-0.5949, -0.7458,  0.3910,  0.3924],
                      [ 0.4012,  0.4930, -0.6113, -0.6209],
                      [-0.9880,  0.4415, -1.6825,  1.4976],
                      [-0.5949, -0.7458,  0.3910,  0.3924],
                      [ 0.4012,  0.4930, -0.6113, -0.6209],
                      [-0.9880,  0.4415, -1.6825,  1.4976],
                      [-0.5949, -0.7458,  0.3910,  0.3924],
                      [ 0.4012,  0.4930, -0.6113, -0.6209],
                   

In [28]:
%timeit Batch.cat(batches)

142 µs ± 331 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

311 µs ± 1.93 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
